In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
import keras
print(keras.__version__)

In [3]:
import os, glob
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score

In [4]:
# uncomment the following line to prevent some possible errors
!pip install --upgrade --no-cache-dir gdown

In [5]:
# uncomment the following line to download the dataset if you haven't already
!gdown 1-0d315aj7Ai8NNqat65XDvaOcHDcHiUD

In [6]:
# uncomment the following line to unzip the dataset if you didn't do it before
!unzip famous_paintings.zip > /dev/null 2>&1

In [7]:
files = glob.glob('data/*')
df_train = pd.DataFrame({'full_path': files})
files= [os.path.basename(f)for f in files]
painters = [re.sub(r'_\d+\.jpg$', '', os.path.basename(f)) for f in files]

  # TODO: get the painters' names from the file names
df_train['painter'] = painters
df_train.head()

In [8]:
df_train.head()

In [9]:
# get the list of unique painters in the dataset
class_names = df_train.painter.unique()
print(class_names)

In [10]:
fig, axes = plt.subplots(3, 4, figsize=(15, 15))
random_indices = np.random.choice(df_train.index, size=12, replace=False)
for i, ax in zip(random_indices, axes.flatten()):
    img = keras.preprocessing.image.load_img(df_train.full_path[i], target_size=(224, 224))
    ax.imshow(img)
    ax.set_title(df_train.painter[i])
    ax.axis('off')
plt.tight_layout()
plt.show()

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['label'] = le.fit_transform(df_train['painter'])# TODO: encode the painters' names
df_train.head()

In [12]:
print(df_train['label'].unique().min())
print(df_train['label'].unique().max())

In [13]:
import shutil

# create a folder for each label
for label in df_train.label.unique():
    os.makedirs(f'data/{label}', exist_ok=True)

# move each image to its corresponding label folder
for i, row in df_train.iterrows():
    shutil.move(row.full_path, f'data/{row.label}')

In [14]:
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory = './data/', # TODO
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',# TODO
    batch_size=32, # TODO
    image_size=(224, 224), # TODO
    shuffle=True,
    validation_split=0.2,
    subset='training',# TODO
    seed=42
)

# find the class names so in prediction time we can map the predictions to the painters properly
class_names = train_dataset.class_names
# print('Class names:', class_names)

val_dataset = image_dataset_from_directory(
    directory = './data/', # TODO
    labels = 'inferred',
    label_mode = 'categorical',
    color_mode = 'rgb',# TODO
    batch_size=32, # TODO
    image_size=(224, 224), # TODO
    shuffle=False,
    validation_split=0.2,
    subset='validation',# TODO
    
)

In [15]:
from keras.applications.resnet50 import preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
# Preprocess the data
val_dataset = val_dataset.map(lambda x,y:(preprocess_input(x),y)) # TODO: apply the preprocess_input function to the val_dataset

In [16]:
import plotly.express as px

# metric: 'accuracy' or 'loss'
def display_curves(history, metric):
  df = pd.DataFrame(history.history[metric], columns=[metric])
  df['val_'+metric] = history.history['val_'+metric]
  fig = px.line(df, x= df.index+1, y= [metric, 'val_'+metric])
  fig.update_layout(xaxis_title='Epochs', yaxis_title=metric)
  fig.show()

In [17]:
from tensorflow import keras
from keras.layers import RandomBrightness, RandomFlip, RandomRotation, RandomZoom, RandomContrast

def build_model(use_pretrained=True, fine_tune_layers=None, use_data_augmentation=False, input_shape=(224, 224, 3), num_classes=10):
    # Data Augmentation
    if use_data_augmentation:
        data_augmentation = keras.Sequential([
            RandomBrightness(0.3),
            RandomFlip("horizontal"),
            RandomRotation(0.2),
            RandomZoom(0.1),
            RandomContrast(0.1)
        ])
    else:
        data_augmentation = None

    # Base Model
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights='imagenet' if use_pretrained else None,
        input_shape=input_shape
    )

    if fine_tune_layers is None:
        base_model.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
        for layer in base_model.layers[-fine_tune_layers:]:
            layer.trainable = True

    # Classification Head
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(128, activation='relu',keras.regularizers.l1_l2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=output)
    return model, data_augmentation

In [18]:
from tensorflow import keras
from keras.layers import RandomBrightness, RandomFlip, RandomRotation, RandomZoom, RandomContrast

def build_model(use_pretrained=True, fine_tune_layers=None, use_data_augmentation=False, input_shape=(224, 224, 3), num_classes=10):
    # Data Augmentation
    if use_data_augmentation:
        data_augmentation = keras.Sequential([
            RandomBrightness(0.3),
            RandomFlip("horizontal"),
            RandomRotation(0.2),
            RandomZoom(0.1),
            RandomContrast(0.1)
        ])
    else:
        data_augmentation = None

    # Base Model
    base_model = keras.applications.ResNet50(
        include_top=False,
        weights='imagenet' if use_pretrained else None,
        input_shape=input_shape
    )

    if fine_tune_layers is None:
        base_model.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
        for layer in base_model.layers[-fine_tune_layers:]:
            layer.trainable = True

    # Classification Head
    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(l1=1e-4, l2=1e-5))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l1_l2(l1=1e-4, l2=1e-5))(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=output)
    return model, data_augmentation

In [19]:
# Build Model
model, data_aug = build_model(
    use_pretrained=True,
    fine_tune_layers=30,
    use_data_augmentation=True,
    num_classes=len(class_names)
)

# # Apply Data Augmentation
# if data_aug:
#     train_dataset = train_dataset.map(lambda x, y: (data_aug(x, training=True), y))

# Compile
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
history = model.fit(train_dataset, epochs=30, validation_data=val_dataset, callbacks=[lr_scheduler])

In [20]:
# Build Model
model, data_aug = build_model(
    use_pretrained=True,
    fine_tune_layers=30,
    use_data_augmentation=True,
    num_classes=len(class_names)
)

# # Apply Data Augmentation
# if data_aug:
#     train_dataset = train_dataset.map(lambda x, y: (data_aug(x, training=True), y))

# Compile
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[lr_scheduler])

In [21]:
display_curves(history, 'loss')

In [22]:
display_curves(history, 'accuracy')

In [23]:
# ارزیابی روی داده‌های اعتبارسنجی
val_loss, val_accuracy = model.evaluate(val_dataset)
test_dataset = image_dataset_from_directory(
    directory = './test_data',
    labels = None,
    image_size=(224, 224),
    color_mode='rgb',
    shuffle=False
)
test_dataset = test_dataset.map(lambda x: preprocess_input(x))

test_predictions = model.predict(test_dataset)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

In [24]:
# Build Model
model, data_aug = build_model(
    use_pretrained=True,
    fine_tune_layers=30,
    use_data_augmentation=True,
    num_classes=len(class_names)
)

# # Apply Data Augmentation
# if data_aug:
#     train_dataset = train_dataset.map(lambda x, y: (data_aug(x, training=True), y))

# Compile
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=[lr_scheduler])

In [25]:
model.save('Best_Model.h5')
# model = keras.models.load_model('my_model.h5')

In [26]:
# history = model.fit(train_dataset, epochs=10, validation_data=val_dataset) # TODO

In [27]:
display_curves(history, 'loss')

In [28]:
display_curves(history, 'accuracy')

In [29]:
# ارزیابی روی داده‌های اعتبارسنجی
val_loss, val_accuracy = model.evaluate(val_dataset)
test_dataset = image_dataset_from_directory(
    directory = './test_data',
    labels = None,
    image_size=(224, 224),
    color_mode='rgb',
    shuffle=False
)
test_dataset = test_dataset.map(lambda x: preprocess_input(x))

test_predictions = model.predict(test_dataset)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

In [30]:

# Load filenames (without .jpg)
file_paths = glob.glob('test_data/*.jpg')
file_names = [os.path.splitext(os.path.basename(f))[0] for f in file_paths]

# Create submission DataFrame
submission = pd.DataFrame()
submission['file'] = file_names

# Get predictions (assumes test_predictions already created)
predicted_indices = np.argmax(test_predictions, axis=1)
# predicted_indices → e.g., [0, 3, 2, 5, 1]
# class_names → ['Andy_Warhol', 'Claude_Monet', ..., 'Vincent_van_Gogh']
predicted_names = [df_train['painter'][i] for i in predicted_indices]
# predicted_names → ['Andy Warhol', 'Claude Monet', ..., 'Vincent van Gogh']
submission['artist'] = predicted_names  # ← ready to go

print(submission)